# Utilities

In [ ]:
from brokenaxes import brokenaxes
from datetime import datetime
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import os
import pandas as pd
import seaborn as sns
import subprocess
import time
import warnings
warnings.filterwarnings('ignore')


source_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
project_mount_point = '/tmp/skyrise'
aws_access_key_id_command = ["aws", "--profile", "default", "configure", "get", "aws_access_key_id"]
aws_access_key_id = subprocess.check_output(aws_access_key_id_command, text=True).strip()
aws_secret_access_key_command = ["aws", "--profile", "default", "configure", "get", "aws_secret_access_key"]
aws_secret_access_key = subprocess.check_output(aws_secret_access_key_command, text=True).strip()

def json_to_dataframe(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    runs = data['runs']
    data_list = []

    for run in runs:
        parameters = run['parameters']
        repetitions = run['repetitions']

        for rep in repetitions:
            invocations = rep['invocations']
            throughput_upload = rep['aggregated_throughput_upload_mbps']
            throughput_download = rep['aggregated_throughput_download_mbps']

            for inv in invocations:
                record = {
                    'aggregated_throughput_upload_mbps': throughput_upload,
                    'aggregated_throughput_download_mbps': throughput_download,
                    'intervals_upload': inv['intervals_upload'],
                    'intervals_download': inv['intervals_download'],
                    'concurrent_instance_count': parameters['concurrent_instance_count'],
                    'report_interval_ms': parameters['report_interval_ms']
                }
                data_list.append(record)

    df = pd.DataFrame(data_list)
    return df

# Network Bursting

## Common Network Throuhgput Measurement


In [ ]:
palette = ['#029e73', '#cc78bc']
sns.set_theme(context="notebook", style="whitegrid", palette=palette, font="Times New Roman", font_scale=1.5)

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
output_file = f"cmake-build-release/bin/{timestamp}_concurrent_instance_counts=1_report_interval_ms=1000.json"

docker_command = [
    "docker", "run", "--rm",
    "--volume", f"{source_dir}:{project_mount_point}",
    "-e" f"AWS_ACCESS_KEY_ID={aws_access_key_id}",
    "-e" f"AWS_SECRET_ACCESS_KEY={aws_secret_access_key}",
    "hpides/skyrise:al2023-arm-20250608",
    f"{project_mount_point}/cmake-build-release/bin/lambdaNetworkBenchmark",
    "--function_instance_sizes_mb=7076",
    "--concurrent_instance_counts=8",
    "--repetition_count=1",
    "--duration_s=10",
    "--report_interval_ms=1000",
    "--enable_download",
    "--enable_upload",
    f"{project_mount_point}/{output_file}"
]

try:
    subprocess.run(docker_command, check=True)
    print("Success.")
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")
    exit(1)

time.sleep(2)

json_file = f"{source_dir}/{output_file}"
# json_file = '../../../resources/benchmark/measurement/network_throughput/lambda/20250323_concurrent_instance_counts=1_report_interval_ms=1000.json'

conversion_factor_mib = 0.95367
df = json_to_dataframe(json_file)
upload_intervals = np.concatenate(df['intervals_upload'].values)
upload_intervals_new = [value * conversion_factor_mib for value in upload_intervals]
download_intervals = np.concatenate(df['intervals_download'].values)
download_intervals_new = [value * conversion_factor_mib for value in download_intervals]

linewidth=1.5
fig, ax = plt.subplots()
ax.plot(np.arange(len(upload_intervals_new)), upload_intervals_new, label="Outbound", linewidth=linewidth)
ax.plot(np.arange(len(download_intervals_new)), download_intervals_new, label="Inbound", linewidth=linewidth)
ax.set_xlabel('Time [seconds]')
ax.set_xlim(0, 8.99)
ax.set_ylabel('Throughput [MiB/s]', labelpad = 30)
ax.set_ylim(0, 399)
ax.set_yticks([0, 100, 200, 300])
for spine in ["top", "right"]:
    ax.spines[spine].set_visible(False)
ax.legend(loc=9, bbox_to_anchor=(0.88, 1.015), ncol=1)

plt.savefig("4_2_network_bursting_configuration_1000ms.pdf", bbox_inches="tight")
plt.show()



## Measurement with Fine Intervals (= 20ms)

In [ ]:
sns.set_theme(context="notebook", style="whitegrid", font="Times New Roman", font_scale=1.5)

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
output_file = f"cmake-build-release/bin/{timestamp}_concurrent_instance_counts=1_report_interval_ms=20.json"

docker_command = [
    "docker", "run", "--rm",
    "--volume", f"{source_dir}:{project_mount_point}",
    "-e" f"AWS_ACCESS_KEY_ID={aws_access_key_id}",
    "-e" f"AWS_SECRET_ACCESS_KEY={aws_secret_access_key}",
    "hpides/skyrise:al2023-arm-20250608",
    f"{project_mount_point}/cmake-build-release/bin/lambdaNetworkBenchmark",
    "--function_instance_sizes_mb=7076",
    "--concurrent_instance_counts=1",
    "--repetition_count=2",
    "--duration_s=1",
    "--report_interval_ms=20",
    "--enable_download",
    "--enable_upload",
    f"{project_mount_point}/{output_file}"
]

try:
    subprocess.run(docker_command, check=True)
    print("Success.")
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")
    exit(1)

time.sleep(2)

json_file = f"{source_dir}/{output_file}"
# json_file = '../../../resources/benchmark/measurement/network_throughput/lambda/20250323_concurrent_instance_counts=1_report_interval_ms=20.json'

report_interval_ms=20
duration_ms=1000
observation_count=int(duration_ms/report_interval_ms)
conversion_factor_gib = 0.95367 / 1000

sleep_array = np.full(
  shape=observation_count,
  fill_value=-1,
  dtype=int
)
df = json_to_dataframe(json_file)
sleep_row = {'intervals_upload': sleep_array,
            'intervals_download': sleep_array,
            'concurrent_instance_count': 1,
            'report_interval_ms': report_interval_ms}
part1 = df.iloc[:1]
part2 = df.iloc[1:]
df = pd.concat([part1, pd.DataFrame([sleep_row]), part2], ignore_index=True)

upload_intervals = np.concatenate(df['intervals_upload'].values)
download_intervals = np.concatenate(df['intervals_download'].values)
upload_intervals = np.insert(upload_intervals,0, -1)
upload_intervals_new = [value * conversion_factor_gib for value in upload_intervals]
download_intervals_new = [value * conversion_factor_gib for value in download_intervals]
upload_intervals_shift = upload_intervals_new[:-1]
download_intervals = download_intervals_new

linewidth=1.5
bax = brokenaxes(xlims=((0, observation_count), (observation_count*2, len(upload_intervals_shift))),  wspace=0.075)
bax.plot(np.arange(len(upload_intervals_shift)), upload_intervals_shift, label='Outbound', linewidth=linewidth, color='#029e73')
bax.plot(np.arange(len(download_intervals)), download_intervals, label='Inbound', linewidth=linewidth,color='#cc78bc')
bax.set_xlabel('Time [milliseconds] [broken axis]',labelpad = 30)
bax.set_ylabel('Throughput [GiB/s]',labelpad = 50)
bax.legend(loc=9, bbox_to_anchor=(0.88, 1.015), ncol=1)
bax.grid(True)
bax.set_ylim(0, 2.4)
bax.axvline(x=observation_count, color='black', linestyle='--', linewidth=2)
bax.axvline(x=observation_count*2, color='black', linestyle='--', linewidth=2)
bax.annotate("sleep for 3 seconds", xy=(observation_count, 1.25), xytext=(observation_count+1.88, 1.25), ha='center', va='center', rotation=90, fontstyle='italic')
bax.set_xticklabels(['0', '0', '400','800', '1200'])
bax.axs[1].set_xticklabels(['0', '4000', '4400','4800'])
bax.set_yticks([0, 0.5, 1.0, 1.5, 2.0])
bax.set_yticklabels(['0', '0', '0.5', '1.0', '1.5', '2.0'])

plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.savefig("4_2_network_bursting_configuration.pdf", bbox_inches="tight")
plt.show()
plt.subplots_adjust(top=1)

## Measurement with Very Fine Intervals (= 2ms)

In [ ]:
sns.set_theme(context="notebook", style="whitegrid", font="Times New Roman", font_scale=1.5)

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
output_file = f"cmake-build-release/bin/{timestamp}_concurrent_instance_counts=1_report_interval_ms=2.json"

docker_command = [
    "docker", "run", "--rm",
    "--volume", f"{source_dir}:{project_mount_point}",
    "-e" f"AWS_ACCESS_KEY_ID={aws_access_key_id}",
    "-e" f"AWS_SECRET_ACCESS_KEY={aws_secret_access_key}",
    "hpides/skyrise:al2023-arm-20250608",
    f"{project_mount_point}/cmake-build-release/bin/lambdaNetworkBenchmark",
    "--function_instance_sizes_mb=7076",
    "--concurrent_instance_counts=1",
    "--repetition_count=2",
    "--duration_s=1",
    "--report_interval_ms=2",
    "--enable_download",
    "--enable_upload",
    f"{project_mount_point}/{output_file}"
]

try:
    subprocess.run(docker_command, check=True)
    print("Success.")
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")
    exit(1)

time.sleep(2)

json_file = f"{source_dir}/{output_file}"
# json_file = '../../../resources/benchmark/measurement/network_throughput/lambda/20250323_concurrent_instance_counts=1_report_interval_ms=2.json'

report_interval_ms=2
duration_ms=1000
observation_count=int(duration_ms/report_interval_ms)
conversion_factor_gib = 0.95367 / 1000

sleep_array = np.full(
  shape=observation_count,
  fill_value=-1,
  dtype=int
)
df = json_to_dataframe(json_file)
sleep_row = {'intervals_upload': sleep_array,
            'intervals_download': sleep_array,
            'concurrent_instance_count': 1,
            'report_interval_ms': report_interval_ms}
part1 = df.iloc[:1]
part2 = df.iloc[1:]
df = pd.concat([part1, pd.DataFrame([sleep_row]), part2], ignore_index=True)

upload_intervals = np.concatenate(df['intervals_upload'].values)
download_intervals = np.concatenate(df['intervals_download'].values)
upload_intervals = np.insert(upload_intervals,0, -1)
upload_intervals_new = [value * conversion_factor_gib for value in upload_intervals]
download_intervals_new = [value * conversion_factor_gib for value in download_intervals]
upload_intervals_shift = upload_intervals_new[:-1]
download_intervals = download_intervals_new

linewidth=0.75
bax = brokenaxes(xlims=((0, observation_count), (observation_count*2, len(upload_intervals_shift))),  wspace=0.075)
bax.plot(np.arange(len(upload_intervals_shift)), upload_intervals_shift, label='Outbound', linewidth=linewidth, color='#029e73')
bax.plot(np.arange(len(download_intervals)), download_intervals, label='Inbound', linewidth=linewidth,color='#cc78bc')
bax.set_xlabel('Time [milliseconds] [broken axis]',labelpad = 30)
bax.set_ylabel('Throughput [GiB/s]',labelpad = 50)
bax.legend(loc=9, bbox_to_anchor=(0.88, 1.015), ncol=1)
bax.grid(True)
bax.set_ylim(0, 3.5)
bax.axvline(x=observation_count, color='black', linestyle='--', linewidth=2)
bax.axvline(x=observation_count*2, color='black', linestyle='--', linewidth=2)
bax.annotate("sleep for 3 seconds", xy=(observation_count, 1.75), xytext=(observation_count+18.8, 1.75), ha='center', va='center', rotation=90, fontstyle='italic')
bax.set_xticklabels(['0', '0', '400','800', '1200'])
bax.axs[1].set_xticklabels(['0', '4000', '4400','4800'])

plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.savefig("4_2_network_bursting_configuration_2ms.pdf", bbox_inches="tight")
plt.show()
plt.subplots_adjust(top=1)

## Scalability of Burst and Baseline Throughput

In [ ]:
palette = ['#0173B2', '#DE8F05', '#029E73', '#D55E00', '#CC78BC', '#ECE133']
sns.set_theme(context="notebook", style="whitegrid", palette=palette, font="Times New Roman", font_scale=1.5)

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
output_file = f"cmake-build-release/bin/{timestamp}_concurrent_instance_counts=10.json"

docker_command = [
    "docker", "run", "--rm",
    "--volume", f"{source_dir}:{project_mount_point}",
    "-e" f"AWS_ACCESS_KEY_ID={aws_access_key_id}",
    "-e" f"AWS_SECRET_ACCESS_KEY={aws_secret_access_key}",
    "hpides/skyrise:al2023-arm-20250608",
    f"{project_mount_point}/cmake-build-release/bin/lambdaNetworkBenchmark",
    "--function_instance_sizes_mb=7076",
    "--concurrent_instance_counts=8",
    "--repetition_count=1",
    "--duration_s=1",
    "--report_interval_ms=20",
    "--enable_download",
    "--enable_upload",
    f"{project_mount_point}/{output_file}"
]

try:
    subprocess.run(docker_command, check=True)
    print("Success.")
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")
    exit(1)

time.sleep(2)

json_file = f"{source_dir}/{output_file}"
json_files = [
    '../../../cmake-build-release/bin/20250325140845_concurrent_instance_counts=1.json',
    '../../../cmake-build-release/bin/20250325141035_concurrent_instance_counts=2.json',
    '../../../cmake-build-release/bin/20250325143637_concurrent_instance_counts=4.json',
    '../../../cmake-build-release/bin/20250325144654_concurrent_instance_counts=6.json',
    '../../../cmake-build-release/bin/20250325143544_concurrent_instance_counts=8.json',
    json_file
    ]
legend = ['1', '2', '4', '6', '8', '10']

index = 0
linewidth=2
for json_file in json_files:
    df = json_to_dataframe(json_file)
    download_intervals = np.concatenate(df['aggregated_throughput_download_mbps'].values)
    download_intervals_gib = (download_intervals / 1024)
    download_intervals_gib_first = download_intervals_gib[:50]

    plt.plot(np.arange(len(download_intervals_gib_first)),
             download_intervals_gib_first,
             label=legend[index],
             linewidth=linewidth,
             color=palette[index])

    index = index +1

plt.xlabel('Time [milliseconds]')
plt.ylabel('Throughput [GiB/s]')
legend = plt.legend(loc='best', ncol=2, title="Concurrent Invocations")
legend._legend_box.align = "left"
plt.xlim(0, 50)
ax = plt.gca()
for spine in ["top", "right"]:
    ax.spines[spine].set_visible(False)
plt.gca().set_xticklabels([0, 200, 400, 600, 800, 1000])

plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.show()

## Scalability of Throughput in VPCs

In [ ]:
files_vpc = [
    '../../../resources/benchmark/measurement/network_throughput/lambda/28032024_concurrent_instance_counts=32_enable_vpc=true.json',
    '../../../resources/benchmark/measurement/network_throughput/lambda/28032024_concurrent_instance_counts=64_enable_vpc=true.json',
    '../../../resources/benchmark/measurement/network_throughput/lambda/28032024_concurrent_instance_counts=128_enable_vpc=true.json',
    '../../../resources/benchmark/measurement/network_throughput/lambda/28032024_concurrent_instance_counts=256_enable_vpc=true.json']

files_no_vpc = [
    '../../../resources/benchmark/measurement/network_throughput/lambda/28032024_concurrent_instance_counts=32_enable_vpc=false.json',
    '../../../resources/benchmark/measurement/network_throughput/lambda/28032024_concurrent_instance_counts=64_enable_vpc=false.json',
    '../../../resources/benchmark/measurement/network_throughput/lambda/28032024_concurrent_instance_counts=128_enable_vpc=false.json',
    '../../../resources/benchmark/measurement/network_throughput/lambda/28032024_concurrent_instance_counts=256_enable_vpc=false.json']

linewidth=2
report_interval_ms=20
duration_ms=1000
observation_count=int(duration_ms/report_interval_ms)
fontsize=1
sns.set_palette("colorblind")
sns.set_context("paper")
sns.set_theme(rc={'figure.figsize':(8.9,5.5)})
sns.set(style="whitegrid", font="Times New Roman", font_scale=1.5)
colors = ['#0173B2', '#DE8F05', '#029E73', '#D55E00', '#CC78BC']
legend_vpc = ['32', '64', '128', '256']
index = 0
for file in files_no_vpc:
    df = json_to_dataframe(file)
    download_intervals = np.concatenate(df['aggregated_throughput_download_mbps'].values)
    download_intervals_gib = (download_intervals / 1024)
    download_intervals_gib_first = download_intervals_gib[:51]

    plt.plot(np.arange(len(download_intervals_gib_first)),
             download_intervals_gib_first,
             label=legend_vpc[index],
             linewidth=linewidth,
             color=colors[index])

    index = index +1

legend_no_vpc = ['32 VPC', '64 VPC', '128 VPC', '256 VPC']
index = 0
for file in files_vpc:
    df = json_to_dataframe(file)
    download_intervals = np.concatenate(df['aggregated_throughput_download_mbps'].values)
    download_intervals_gib = (download_intervals / 1024)
    download_intervals_gib_first = download_intervals_gib[:51]
    # Plotting the line plot
    plt.plot(np.arange(len(download_intervals_gib_first)), download_intervals_gib_first, label=legend_no_vpc[index], linewidth=linewidth, linestyle='dashdot', color=colors[index])
    index = index +1

# Adding labels and title
plt.yscale('log')
plt.xlabel('Time [milliseconds]')
plt.ylabel('Throughput [GiB/s] [log scale]')
legend = plt.legend(loc='best', ncol=2, title="Concurrent Invocations")
legend._legend_box.align = "left"
plt.xlim(0, 50)
#plt.xticks(np.arange(0, 50, step=10))
ax = plt.gca()
ax.set_yscale('log')
ax.set_yticks([0, 1, 5, 25, 125, 250])
ax.get_yaxis().set_major_formatter(ticker.ScalarFormatter())
plt.ylim(0, 250)
plt.gca().set_xticklabels([0, 200, 400, 600, 800, 1000])
custom_ticks = [0,1,5,25,125,250]

plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.savefig("4_2_network_bursting_concurrency.pdf", bbox_inches="tight")
plt.show()


# EC2 Network Measurement

In [ ]:
sleep_s = 1800
files_per_type = [
    ['../../../resources/benchmark/measurement/network_throughput/ec2/repetition=1_instance_type=c6gd.medium_refill_sleep_seconds=1800.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=2_instance_type=c6gd.medium_refill_sleep_seconds=1800.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=1_instance_type=c6gd.medium_refill_sleep_seconds=900.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=2_instance_type=c6gd.medium_refill_sleep_seconds=900.txt'],
    ['../../../resources/benchmark/measurement/network_throughput/ec2/repetition=1_instance_type=c6gd.large_refill_sleep_seconds=1800.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=2_instance_type=c6gd.large_refill_sleep_seconds=1800.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=1_instance_type=c6gd.large_refill_sleep_seconds=900.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=2_instance_type=c6gd.large_refill_sleep_seconds=900.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=1_instance_type=c6g.large_refill_sleep_seconds=300.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=2_instance_type=c6g.large_refill_sleep_seconds=300.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=3_instance_type=c6g.large_refill_sleep_seconds=300.txt'],
    ['../../../resources/benchmark/measurement/network_throughput/ec2/repetition=1_instance_type=c6gd.xlarge_refill_sleep_seconds=1800.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=2_instance_type=c6gd.xlarge_refill_sleep_seconds=1800.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=1_instance_type=c6gd.xlarge_refill_sleep_seconds=900.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=2_instance_type=c6gd.xlarge_refill_sleep_seconds=900.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=1_instance_type=c6g.xlarge_refill_sleep_seconds=300.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=2_instance_type=c6g.xlarge_refill_sleep_seconds=300.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=3_instance_type=c6g.xlarge_refill_sleep_seconds=300.txt'],
    ['../../../resources/benchmark/measurement/network_throughput/ec2/repetition=1_instance_type=c6gd.2xlarge_refill_sleep_seconds=1800.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=2_instance_type=c6gd.2xlarge_refill_sleep_seconds=1800.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=1_instance_type=c6gd.2xlarge_refill_sleep_seconds=1800.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=2_instance_type=c6gd.2xlarge_refill_sleep_seconds=900.txt'],
    ['../../../resources/benchmark/measurement/network_throughput/ec2/repetition=1_instance_type=c6gd.4xlarge_refill_sleep_seconds=900.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=2_instance_type=c6gd.4xlarge_refill_sleep_seconds=1800.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=1_instance_type=c6gd.4xlarge_refill_sleep_seconds=900.txt',
     '../../../resources/benchmark/measurement/network_throughput/ec2/repetition=2_instance_type=c6gd.4xlarge_refill_sleep_seconds=900.txt']]


factor = 0.11641532182693
categories = ['Medium', 'Large', 'XLarge', '2XLarge', '4XLarge', '8XLarge', 'Lambda']
# Baseline bandwidth are emperically determined.
baseline_bandwidth = [0.0582077, 0.08731149, 0.1455192, 0.291038, 0.582076, 1.5, 0.0745058]
budget_values = []
burst_bandwidths = []
refill_rates = []

sns.set_palette("colorblind")
sns.set_context("paper")
sns.set_theme(rc={'figure.figsize':(8.9,4.125)})
sns.set(style="whitegrid", font="Times New Roman", font_scale=1.5)
sns.set_palette("colorblind")
colors = ['#0173b2', '#c7ebff', '#de8f05', '#feedd0']

def calculateBudget(df, file):
    burst = 0
    baseline = 0
    budgets = [0,0]
    duration = [0,0]
    bursts = [0,0]
    index = 0
    for idx, row in df.iterrows():
        if (burst == 0 and index == 0):
            burst = row['bandwidth']
            budgets[index] += row['bandwidth']
            duration[index] = row['second']
        elif (burst != 0 and row['bandwidth'] > burst * 0.9):
            budgets[index] += row['bandwidth']
            duration[index] = row['second']
        elif (index == 0):
            bursts[index] = burst * factor
            index = index + 1
            burst = 0
        else:
            if (baseline == 0):
                baseline = row['bandwidth']
            elif (row['bandwidth'] > baseline * 1.5 and bursts[index] == 0):
                burst = row['bandwidth']
                bursts[index] = burst * factor
            continue

    print("File=", str(file))
    refill_rate = ((budgets[0] * 4) * factor) / sleep_s
    for i in range(0,2):
        print("i=", i, "Capacity (GiB)=", (budgets[i] * 4) * factor, "Duration=", duration[i],
              "RefillRate=", refill_rate, "BurstBandwidth=", (budgets[i] / duration[i]) * factor, "Burst=", bursts[i])

    return [(budgets[0] * 4) * factor, bursts]

result = []
index = 0
for files in files_per_type:
    result.append([[], []])
    for file in files:
        df = pd.read_csv(file)
        intermediate = calculateBudget(df, file)
        result[index][0].append(intermediate[0])
        result[index][1].append(intermediate[1])
    index = index + 1

# 8XL values
result.append([[], []])
result[index][0].append(0)
result[index][1].append(0)

#Lambda values
result.append([[], []])
result[index+1][0].append(0.279397)
#result[index+1][1].append(1.2)
result[index+1][1].append(0.7755846757894737)
result[index+1][1].append(1.055985167142857)
result[index+1][1].append(0.8338784516666667)
result[index+1][1].append(1.2201412925000002)
result[index+1][1].append(0.9760494560000001)
result[index+1][1].append(0.932212425)

print(burst_bandwidths)
print(result)

bursts_median = [np.median(result[0][1]), np.median(result[1][1]), np.median(result[2][1]), np.median(result[3][1]), np.median(result[4][1]),np.median(result[5][1]),np.median(result[6][1])]
y_min = [np.min(result[0][1]), np.min(result[1][1]), np.min(result[2][1]), np.min(result[3][1]), np.min(result[4][1]), np.median(result[5][1]),np.min(result[6][1])]  # Minimum values
y_max = [np.max(result[0][1]), np.max(result[1][1]), np.max(result[2][1]), np.max(result[3][1]), np.max(result[4][1]),np.median(result[5][1]),np.max(result[6][1])]  # Maximum values

# Calculate the error bars
lower_error = [bursts_median[i] - y_min[i] for i in range(7)]
upper_error = [y_max[i] - bursts_median[i] for i in range(7)]

bars = 3
total_bar_width = 0.65
bar_width = total_bar_width / bars
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
x = np.arange(len(categories))

# Plot the first set of values on the primary y-axis
print("-------")
print(result[1][0])
print([np.median(result[0][0]),np.median(result[1][0]),np.median(result[2][0]),np.median(result[3][0]),np.median(result[4][0])])
ax1.bar(x - bar_width, [np.mean(result[0][0]),np.mean(result[1][0]),np.mean(result[2][0]),np.mean(result[3][0]),np.mean(result[4][0]),
                        np.mean(result[5][0]),np.mean(result[6][0])],
         width=bar_width, label='Burst Capacity', color=colors[0])
ax1.set_xlabel('Instance Type')
ax1.set_ylabel('Burst Capacity [GiB]')
ax1.tick_params('y')

# Plot the second set of values on the secondary y-axis
ax2.bar(x, bursts_median, yerr=[lower_error, upper_error], capsize=5, width=bar_width, label='Burst Bandwidth', color=colors[1])
ax2.bar(x + bar_width, baseline_bandwidth, width=bar_width, label='Baseline Bandwidth', color=colors[2])
ax2.set_ylabel('Bandwidth [GiB/s]', labelpad=10)

ax2.tick_params('y')

plt.text(23 * bar_width, 0.032, 'x', ha='center', va='center', color='black', family="monospace", fontsize=19)
plt.text(22 * bar_width, 0.032, 'x', ha='center', va='center', color='black', family="monospace", fontsize=19)

# Set the x-axis ticks and labels
ax1.set_xticks(x, labels=categories)

plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True

ax1.grid(True)
ax2.grid(False)
ax1.set_yscale('log')

fig.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.51, 1.00))
fig = plt.gcf()

plt.savefig('4_2_network_bursting_ec2.pdf', bbox_inches="tight")
plt.show()
